In [2]:
import tensorflow as tf
import numpy as np


In [41]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [42]:
x_train = x_train-np.mean(x_train,axis=0)
x_test = x_test - np.mean(x_test,axis=0)

In [43]:
x_train.shape

(50000, 32, 32, 3)

In [ ]:
from tensorflow.keras.layers import Dense, Conv2D, GlobalAveragePooling2D, BatchNormalization
input_shape = (32,32,3)
"""
For training on cifar10/cifar100 data
create a plain network model, 6n+2 layers,
"""
def create_model(input_shape,n):
    model = tf.keras.Sequential()
    model.add(Conv2D(filters=16,kernel_size=(3, 3),input_shape=input_shape,activation='relu'
             ,kernel_initializer=tf.keras.initializers.HeNormal(),padding="same",use_bias=True))
    for i in range(2*n):
        model.add(Conv2D(filters=16,kernel_size=(3, 3),activation='relu',padding="same"
                     ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal()))
    model.add(Conv2D(filters=32,kernel_size=(3, 3),activation='relu',strides=(2,2),padding="same"
             ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal()))
    for i in range(2*n-1):
        model.add(Conv2D(filters=32,kernel_size=(3, 3),activation='relu',padding="same"
                     ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal()))
    model.add(Conv2D(filters=64,kernel_size=(3, 3),activation='relu',strides=(2,2),padding="same"
             ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal()))
    for i in range(2*n-1):
        model.add(Conv2D(filters=64,kernel_size=(3, 3),activation='relu',padding="same"
                     ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal()))
    model.add(GlobalAveragePooling2D())
    model.add(BatchNormalization())
    model.add(Dense(10,activation='softmax'))
    return model

In [85]:
model = create_model(input_shape,3)

In [86]:
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [87]:
model.fit(x=x_train, 
          y=y_train,
          batch_size=128,
          epochs=10, 
          validation_data=(x_test, y_test))

Epoch 1/10
391/391 [==============================] - 155s 391ms/step - loss: 2.0297 - accuracy: 0.2342 - val_loss: 2.3100 - val_accuracy: 0.1365
Epoch 2/10
391/391 [==============================] - 145s 372ms/step - loss: 1.7316 - accuracy: 0.3498 - val_loss: 1.7867 - val_accuracy: 0.3534
Epoch 3/10
391/391 [==============================] - 151s 386ms/step - loss: 1.6115 - accuracy: 0.4075 - val_loss: 1.8670 - val_accuracy: 0.3226
Epoch 4/10
391/391 [==============================] - 156s 399ms/step - loss: 1.4960 - accuracy: 0.4569 - val_loss: 3.9667 - val_accuracy: 0.1372
Epoch 5/10
391/391 [==============================] - 145s 369ms/step - loss: 1.8167 - accuracy: 0.3052 - val_loss: 2.1053 - val_accuracy: 0.1885
Epoch 6/10
391/391 [==============================] - 142s 364ms/step - loss: 1.7984 - accuracy: 0.3074 - val_loss: 1.8806 - val_accuracy: 0.2939
Epoch 7/10
300/391 [======================>.......] - ETA: 32s - loss: 1.7096 - accuracy: 0.3528

KeyboardInterrupt: 

In [22]:
from tensorflow.keras.layers import Dense, Conv2D, GlobalAveragePooling2D, BatchNormalization
input_shape = (32,32,3)
"""
For training on cifar10/cifar100 data
create a plain network model, 6n+2 layers,
"""
def create_resnet(input_shape,n):
    model = tf.keras.Sequential()
    model.add(Conv2D(filters=16,kernel_size=(3, 3),input_shape=input_shape,activation='relu'
             ,kernel_initializer=tf.keras.initializers.HeNormal(),padding="same",use_bias=True))
    for i in range(2*n):
        model.add(Conv2D(filters=16,kernel_size=(3, 3),activation='relu',padding="same"
                     ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal()))
    model.add(Conv2D(filters=32,kernel_size=(3, 3),activation='relu',strides=(2,2),padding="same"
             ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal()))
    for i in range(2*n-1):
        model.add(Conv2D(filters=32,kernel_size=(3, 3),activation='relu',padding="same"
                     ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal()))
    model.add(Conv2D(filters=64,kernel_size=(3, 3),activation='relu',strides=(2,2),padding="same"
             ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal()))
    for i in range(2*n-1):
        model.add(Conv2D(filters=64,kernel_size=(3, 3),activation='relu',padding="same"
                     ,use_bias=True,kernel_initializer=tf.keras.initializers.HeNormal()))
    model.add(GlobalAveragePooling2D())
    model.add(BatchNormalization())
    model.add(Dense(10,activation='softmax'))
    return model

(10000, 1)